In [19]:
import numpy as np
import scipy.signal as sps
import sympy as sym 
import scipy.optimize as sopt
import scipy.io as sio
%matplotlib inline   
import matplotlib.pyplot as plt

sqrt = np.sqrt
cos = np.cos
pi = np.pi
log10 = np.log10

In [41]:
# Grid parameters
U_lv_b = 690.0
U_m_b = 690.0
V_gpeak = 20e3/sqrt(3)*sqrt(2);
V_g2peak = U_lv_b/sqrt(3)*sqrt(2);
V_m_peak = U_m_b/sqrt(3)*sqrt(2);
f_g = 50;

# AC unit transformer
N_t1 = V_gpeak*sqrt(3); # Delta
N_t2 = V_g2peak;
N_t3 = V_g3peak;
R_t1 = 0.4889/2*3; # Delta
L_tk1 = 0.0118/2*3; # Delta
R_t2 = (0.4889/2)/((V_gpeak/V_g2peak)**2);
L_tk2 = (0.0118/2)/((V_gpeak/V_g2peak)**2);
R_t3 = (0.4889/2)/((V_gpeak/V_g3peak)**2);
L_tk3 = (0.0118/2)/((V_gpeak/V_g3peak)**2);

# PMSM 
P_m_n = 2e6; # MW
rpm = 20
freq_m = 10
omega_e_m = 2*pi*freq_m
N_pp = freq_m/(rpm/60)
X_m_pu = 0.1
Z_m_b = U_lv_b**2/P_n
X_m = X_m_pu*Z_m_b
L_m = X_m/(2.0*pi*freq_m)
P_m_loss = 0.01*P_m_n
I_m_n = P_m_n/(np.sqrt(3.0)*U_lv_b)
R_m = P_m_n/(3*I_m_n**2)
Phi_m = V_m_peak/omega_e_m
V_dc_m = U_m_b*2.0

params_pmsm = {'R_m':R_m,
               'L_m':L_m,
               'Phi_m':Phi_m,
               'omega_e_m':omega_e_m,
               'V_dc_m':V_dc_m,
               'N_pp':N_pp               
              }

# Turbine
H_t = 4.5
H_r = 0.5

# H = 0.5*J*omega**2

J_g = 75; # kg*(m**2) or N*m*(s**2) of DFIG rotor
J_total_hs = J_g/0.1419*(0.6388 + 0.0114 + 0.0806 + 0.1419); # total inertia of WT on high-speed side

# Limitations
I_R_max = 750*sqrt(2)*(1/k_sr)*(1/gamma);

# Machine side converter
# DC-link
C_dc = 53e-3; 
V_dc = 950;
R_brake = 0.13;

# Switching frequency
f_sw = 5e3;

# Flux estimation
w_cutoff = 100;

# Torque control
alpha_c = 100; 
R_mc_a = alpha_c*L_sigma* - R_R - R_s ; # Active damping
K_mc_pd = alpha_c*L_sigma;
K_mc_id = alpha_c*(R_R + R_s + R_mc_a);
K_mc_pq = alpha_c*L_sigma;
K_mc_iq = alpha_c*(R_R + R_s + R_mc_a);

# Speed control
alpha_w = 1;
B_mc_a_w = alpha_w*J_total_hs/n_p;
K_mc_pw = J_total_hs*alpha_w/n_p;
K_mc_iw = B_mc_a_w*alpha_w;
K_mc_b_w = 0.1; # anti-windup

# Reactive power control
alpha_Q = 20;
K_mc_iQ = -alpha_Q/(3 * V_gpeak/N_t1*N_t2*sqrt(3)/2);
K_mc_b_Q = 10; # anti-windup

# Grid side converter
V_g3rms = V_g3peak/sqrt(2);
slip_min = -0.1;
I_g3rms_max = P_n*(-slip_min)/(1-slip_min)/3/V_g3rms;
I_g3peak_max = I_g3rms_max*sqrt(2);
I_g3pp_max = I_g3peak_max*2;

# LCL filter design
THD = 0.04;
I_hfpp_max = 0.2*I_g3peak_max;
L_f1 = V_g3peak/(V_dc/sqrt(3))*cos(pi/6)*(2/3*V_dc - V_g3peak)/(I_hfpp_max*f_sw);
R_f1 = 0;
L_f2 = L_f1*0.15;
C_f = 1/(L_f2*(2*pi*f_sw*10**(20*log10(I_g3pp_max*THD/I_hfpp_max)/40))**2);

# Limitation
I_f_max = 400;

# Current control
alpha_f = 100;
R_res_a = 1e-3; # Active damping resistance for resonance elimitation
R_gc_a = alpha_f*(L_f1 + L_f2) - R_f1;
K_gc_pd = alpha_f*(L_f1 + L_f2);
K_gc_id = alpha_f*(R_f1+R_gc_a);
K_gc_pq = alpha_f*(L_f1 + L_f2);
K_gc_iq = alpha_f*(R_f1+R_gc_a);

# DC-link voltage control
alpha_W = 10;
G_gc_a_W = alpha_W*C_dc/(6 * V_gpeak/N_t1*N_t3*sqrt(3)/2);
K_gc_pW = -alpha_W*C_dc/(6 * V_gpeak/N_t1*N_t3*sqrt(3)/2);  # sqrt(3) for delta-y transformer, /2 for rms value
K_gc_iW = -alpha_W*G_gc_a_W;
K_gc_b = 10; # anti-windup

# Mechanical parameters
k_ls_hs = 100/1; # Gearbox ratio low-speed/high-speed


J_gb = J_g/0.1419*0.0806*(k_ls_hs**2);
J_h = J_g/0.1419*0.0114*(k_ls_hs**2);
J_b = J_g/0.1419*(0.6388/3)*(k_ls_hs**2);

k_gbg = 1834.1*(P_n * (n_p**2))/(f_g*2*pi);
k_hgb = 54.75*(P_n * (n_p**2) * (k_ls_hs**2))/(f_g*2*pi);
k_hb = 1259/3*(P_n * (n_p**2) * (k_ls_hs**2))/(f_g*2*pi);

D_g = 0.01*P_n/((f_g*2*pi/n_p)**2);
D_gb = 0.022*(P_n * (k_ls_hs**2))/((f_g*2*pi/n_p)**2);
D_h = 0.01*(P_n * (k_ls_hs**2))/((f_g*2*pi/n_p)**2);
D_b = 0.004/3*(P_n * (k_ls_hs**2))/((f_g*2*pi/n_p)**2);
D_total_hs = D_g + D_gb/(k_ls_hs**2) + D_h/(k_ls_hs**2) + 3*D_b/(k_ls_hs**2);

d_gbg = 10*P_n/((f_g*2*pi/n_p)**2);
d_hgb = 3.5*(P_n * (k_ls_hs**2))/((f_g*2*pi/n_p)**2);
d_hb = 12/3*(P_n * (k_ls_hs**2))/((f_g*2*pi/n_p)**2);

# Intial conditions
Omega_r_0 = f_g*2*pi/n_p;
Psi_sd_0 = 1e-3;
Psi_sq_0 = 0;# V_gpeak/sqrt(2)*N_t2/N_t1/(2*pi*f_g);

# Wind behavior
TmLookupData = sio.loadmat('TmLookupData.mat');

In [42]:
eta_m_d,eta_m_q = sym.symbols('eta_m_d,eta_m_q')
i_m_d,i_m_q,v_m_dc,i_m_dc = sym.symbols('i_m_d,i_m_q,v_m_dc,i_m_dc')
L_m,R_m,omega_m,Phi_m = sym.symbols('L_m,R_m,omega_m,Phi_m')
J_m_t,J_m_r,K_m_tr,K_d_m_tr = sym.symbols('J_m_t,J_m_r,K_m_tr,K_d_m_tr')


eta_2_d,eta_2_q = sym.symbols('eta_2_d,eta_2_q')
i_2_d,i_2_q,v_2_dc,i_2_dc = sym.symbols('i_2_d,i_2_q,v_2_dc,i_2_dc')
v_2_sd,v_2_sq = sym.symbols('v_2_sd,v_2_sq')
L_2,R_2,C_2,omega_2 = sym.symbols('L_2,R_2,C_2,omega_2')

R_dc = sym.symbols('R_dc')

v_th_d,v_th_q = sym.symbols('v_th_d,v_th_q')
R_th,L_th = sym.symbols('R_th,L_th')
R_12,L_12 = sym.symbols('R_12,L_12')

p_1,q_1,p_2,q_2 = sym.symbols('p_1,q_1,p_2,q_2')

In [43]:
N_pp

30.0

In [44]:
omega_e_m = N_pp
di_m_d  = 1/L_m*(eta_m_d*v_m_dc/2 - R_m*i_m_d - L_m*omega_e_m*i_m_q )
di_m_q  = 1/L_m*(eta_m_q*v_m_dc/2 - R_m*i_m_q + L_m*omega_e_m*i_m_d - omega_e_m*Phi_m)

f = sym.Matrix([[di_m_d],[di_m_q]])
x = sym.Matrix([[i_m_d],[i_m_q]])

F_x = f.jacobian(x)